# Express.js 101

The back end can be a little mysterious because everything's sort of abstract. A server isn't really something you can see, which makes developing/debugging it sort of like grasping at the abyss. Hopefully this tutorial will make it less confusing.

Express.js is a library of functions that makes developing the backend a lot more formal. It lets you create server end points, like an endpoint that receives POST requests on the path "api/courses/search" that returns json data. We'll learn how to create/design/test these endpoints.

## Getting Started

Tips before we start:
- If you haven't already, install a formatter extension for Javascript code into VSCode (if that's what you're using). Either the Prettier or official Javascript/Typescript Nightly extensions will do. Just make sure you DON'T enable "format on save," which can be nice at first but will become a huge pain as you keep developing.
- Rewatch the Javascript tutorial to familiarize yourself with the syntax/features. I expect you to know Javsacript at least decently well going into this.
- You might notice that some of the things you'll see in this tutorial are different than what you're used to in regular Javascript (like how imports are handled), and that's because we're using a special compiler called Node.js to compile our code, which added/replaced certain features.

Let's start by looking at the current state of the project. Since I can't guarantee that the current state of the develop branch is in a consistent one, go ahead and checkout the main branch. If you're using Github Desktop, click as highlighted below and click on the main branch.


![main](../githubcheckout.PNG)

To mess around and try things out, we have to start up the development server too. Open up a terminal in the root directory of the project (which you can due using the New Terminal option in VSCode), then run "npm run debug".

In [ ]:
npm start

That should open up the server at the url localhost:8080. If you try to open up this url in a browser, you'll find that you get a File not found error. I'll get to why this happens later (tl:dr, we haven't compiled the client). Every time you make changes and save the code while in development mode, the server will restart for you, which is pretty neat when you want to make a lot of changes and see how the server reacts.

## Project Overview & Express.js Tutorial

If you feel like you're pretty comfortable with Express.js/the current state of the project and want to get straight to development, jump down to the Development section below.

Now I told you to start the development server, but what did that actually do? "npm run debug" is a command that the npm program (which you installed when you install Node.js) handles like so: look inside inside of the current directory for a package.json file, go to the scripts section, and run the command called "debug".

If you've opened up the client directory before, you'll notice that that folder has a package.json file too, but that's the one for the client. That's why we want to make sure we're in the root folder when we run this command. Let's look at the package.json in the root folder, and just at the scripts section.

In [ ]:
{
  "scripts": {
    "debug": "nodemon server/index.js",
    "start": "node server/index.js",
    "test": "jest server"
  },
}

You'll notice that there's a couple of scripts here. The debug script (the one we use to run the development server) is very close to the start script. That's because they both run the same file, server/index.js, but they use different programs to do so. The "node" program is the default node compiler that runs javascript code and comes with Node.js. It's kind of like running "python index.py". "nodemon" is a program that I installed, and it's actually one of the development dependencies that you can find lower in the package.json file. The nodemon program is what reloads the server every time our code changes. From these scripts, we can identify that the first piece of code that is run when we start the server is at the path server/index.js. Let's look at that file.

In [ ]:
// import the app
const app = require('./server');
// pick a port number
const port = process.env.PORT || 8080;

// start the server
app.listen(port, () => console.log(`Server Start! Listening on port ${port}`));


Looks simple enough, with maybe some features of Javascript you haven't seen before. The first line is basically an import statement from Python: "import server as app". Notice how, just like in Python, you don't have to include the .js file extension when you're importing from that file. We'll look at the server.js file in a second, but you might be a little confused about what line 4 does:

In [ ]:
const port = process.env.PORT || 8080;

For now, don't worry too much about it, but it basically says "hey, does process.env.PORT return null? If not, set the port to process.env.PORT. If it is null, set the port to 8080."

After that, the server starts at the port designated by the port variable, and once it's done, it calls that callback function that prints "Server Start!..." to the console to let us know that the server has started.

Now let's dig into server.js, which starts with a bunch of imports.

## Imports

In [ ]:
const express = require("express");
const dotenv = require("dotenv");
const bodyParser = require("body-parser");
const cors = require("cors");
const path = require("path");
const routes = require("./routes"); // import routes from routes/index.js

Notice how there's sort of 2 categories of imports (ignore the highlighting, I'm talking about local/non-local exports). The imports that don't start with a dot (".") are libraries that we installed, and the import that does start with a dot is actually a folder.

If you've ever come across this feature in Python, if you write "require("./routes")" in Javascript, it will check to see if ./routes is a Javascript file. If it's a folder instead, it will try to import ./routes/index.js. If that doesn't work it'll bug out. Luckily we do have a ./routes/index.js file, but we'll get into that later.

In [ ]:
// Load environment variables from .env file
dotenv.config({ path: __dirname + "/.env" });

What does the statement above do? It basically sets up environment variables, which you can read about here: https://www.npmjs.com/package/dotenv
These aren't super important yet, but if you remember line 4 from index.js, "process.env.PORT" would be defined from the environment variables. Let's keep going in server.js

In [ ]:
// create the application
const app = express();

// apply middleware
app.use(bodyParser.json());
app.use(cors());

Here we create the application and apply some middleware. What is the app? It's literally everything in Express.js. It's what you hook up middleware to. It's what you hook up server endpoints to. It's what you start to get the server running. This object is the central point to how we handle the server, which means if we want to make changes to the server, we just pass around the app.

## Middleware

What's middleware? It's a little hard to describe, but think of it as what goes in between frontend and backend. That's a little ambiguous, so I'll just explain what each of these middleware does. The bodyParser middleware (which was installed as an npm package), automatically converts the body of POST requests into JSON objects, and the cors middleware gets around the Cross-Origin Resource Sharing Policy, which basically won't let you receive/send content to another website unless you adhere to its format, and applying the cors middleware does that.

If you look at the current state of the project (on the development branch), you'll notice that I added more middleware, and feel free to look at what those do (they should all be npm packages that you can find the documentation for online).

In [ ]:
// set up the routes for the api and the static files
app.use("/api", routes);
app.use(express.static(path.join(__dirname, "../client/build")));

Here's where the meat of our server is. It hooks up the "routes" object to the "/api" path, which means whatever request we make to the "routes" object has to start with the "/api" path. So, what would have originally been "zotsearch.com/search" is now "zotsearch.com/api/search". We do this to differentiate the API and the website.

What's an API? It has a very generic definition, but in the context of our server, it's the collection of server endpoints that we request JSON data from. In contrast, we have line 20, which is the path to the website file (html) itself.

In [ ]:
app.use(express.static(path.join(__dirname, "../client/build")));

There's a lot going on in this line, so let's break it down piece by piece. "app.use" is something we've been seeing a lot. It basically means, hook up whatever this thing is to our server. "express.static" basically means, the files in this folder should be accessibly by the user. "path.join" joins two path strings, e.g. if you join "/foo" and "/bar", you get "/foo/bar". "__dirname" is a Node.js macro that always returns the folder path that this file is currently in, which in this case is "root/server". The double dots ("..") mean the parent folder of the current folder. If we join "root/server" with "../client/build", we get "root/client/build". 

To put all of that together, this line is basically saying: make all of the files in the "root/client/build" folder accessible to the user. But you'll notice, there is not "build" folder in the "client" folder, and that's because that folder gets generated when we build the production version of the client. I'll show you how to do that in another tutorial when it's more relevant.

In [ ]:
// default route to index.html
app.get("*", (req, res) => {
    res.sendFile(path.join(__dirname, "../client/build", "index.html"));
  });  

This next line basically says, if the user goes to any path other than the ones specified above, send them to the index.html page in the "root/client/build" folder.

In [ ]:
module.exports = app;

This is how we end the file, and, unlike in Python, in Javascript, you have to tell the compiler which variables should be accessible by other files by setting the module.exports. This is a Node.js feature, so it won't work in regular Javascript. Since the app is what our module is exporting, it's exactly what we get when we "require"/import it from index.js.

## Routes

Now let's look at our "routes/index.js" file

In [ ]:
const express = require("express");
const userRoute = require("./user");

// the base router for all api endpoints
const router = express.Router();

// install the user route
router.use("/user", userRoute);

module.exports = router;

Looks simple enough, this time we require from "./user" which is a Javascript file in the routes folder, stick it onto the "/user" path, and bingo bongo we have our router. Now every time we access any path on the userRoute, we have to go through "zotsearch.com/api/user/endpoint" instead of "zotsearch.com/api/endpoint". Let's look at user.js now.

## Endpoints

In [ ]:
const express = require("express");
const router = express.Router();

// dummy endpoint for testing
router.get("/sup", (req, res) => {
  res.send({ foo: "bar" });
});

module.exports = router;

So this is the format of an endpoint. In lines 1 and 2, we import Express.js, and create a router. What's a router? It routes users to endpoints, which is very abstract/vague but lines 5-7 will clear it up hopefully.

In [ ]:
router.get("/sup", (req, res) => {
    res.send({ foo: "bar" });
  });

Let's break this down piece by piece. We start with "router.get", which is a function that takes 2 parameters. "router.get(a, b)" basically means, hey, we're expecting a GET request on the path specified by "a", and when we get a request on that path, call function "b".

Since we're hooking up routers to more routers, this path "/sup" will end up being "/api/user/sup". But let's look at the arrow function we pass into router.get. It takes 2 arguments, req (for request) and res (for response). Let's try logging these to the console like so:

In [ ]:
const express = require("express");
const router = express.Router();

// dummy endpoint for testing
router.get("/sup", (req, res) => {
  console.log(req);
  console.log(res);
  res.send({ foo: "bar" });
});

module.exports = router;

If you save, you'll notice that the server restarted for you, sweet. Now how do you access this endpoint? You need a program that can request data from endpoints and display it somehow. I bet you have one already installed, it's called a browser! Try going to the url: http://localhost:8080/api/user/sup

Let's learn how to read that url. "http://" means that you're making an HTTP request. "localhost" is the host name you're going to, which in this case is your own computer. "8080" is the port that identifies the program we're accessing, and "/api/user/sup" is the path we're accessing on that program. (Something to note, browsers always make GET requests). You'll see in the browser: 

In [ ]:
{"foo":"bar"}

Which is what our endpoint returns, but more importantly, look at what our server log shows. We got some massive objects, maybe more than your console could show. Now, how are we supposed to code with these things? That's what the documentation is for. It tells you everything you need to know about how that framework works. If you go to the API reference for the Request and Response objects in Express.js: https://expressjs.com/en/4x/api.html#req, you'll find everything you need, but I'll provide some pointers as to what's important.

## Requests

Go ahead and paste this code into user.js:

In [ ]:
const express = require("express");
const router = express.Router();

router.get("/sup", (req, res) => {
  console.log(req.query);
  res.send({ foo: "bar" });
});

router.post("/hi", (req, res) => {
  console.log(req.body);
  res.send({ foo: "bar" });
});

module.exports = router;

You'll noticed I added a couple of things. The first thing is a log of "req.query" inside of the "/sup" endpoint. "req.query" is how we will be passing arguments into GET requests.

The second thing I added is another endpoint to the path "/hi". Notice that this one's a POST request endpoint because it uses "router.post". Also notice that it logs "req.body" instead. "req.body" is how we will be passing arguments into POST requests. What's the difference? Let's look at how we can add query arguments to our GET request. Go to the link: http://localhost:8080/api/user/sup?foo=bar

Notice that at the end I added "?foo=bar". Now if you look inside the log for our server, you'll notice that "req.query" got logged as "{foo: bar}". We can add query parameters to our get requests just by sticking things at the end of our url. If you wanted another argument, you could use the & symbol like "?foo=bar&baz=faz". You might ask, can you put lists/dictionaries in there? I supposed you can, but it's highly ill-advised because GET requests have length restrictions for the URL. How should you add lists/dictionaries into your requests then? 

Meet the POST request. If we look at the code that implements the POST request endpoint for "hi", it's not too far off from the GET request example:

In [ ]:
router.post("/hi", (req, res) => {
    console.log(req.body);
    res.send({ foo: "bar" });
  });

In this case, we use "req.body" instead of "req.query", which can be of any length. How do we make a POST request? Typing stuff into the URL bar of our browser is limited to GET requests, so we need some kind of program to run code easily that will make requests for us. I bet you have one installed, it's called...a browser! If you go to the inspector (Ctrl+Shift+I or right click), and switch to the Console Tab, you can run Javascript code right there. Run this line of Javascript code below (don't worry it won't hack you, and I'll go through it piece by piece to prove it).

In [ ]:
fetch("http://localhost:8080/api/user/hi", {
  method: "POST",
  body: JSON.stringify({ x: 1 }),
  headers: { "Content-Type": "application/json" },
});

The fetch function is Javascript makes HTTP requests to the given path, and can take arguments to configure that request. The first one we pass in is the method, in this case "POST". The second is the body of our HTTP request, which our endpoint will receive in "req.body". Notice that it has to be a string, so we call JSON.stringify to turn our object "{ x: 1 }" into a string. We then pass in a header which tells the server that it will be received json data (which is what "application/json" specifies). If you copy+paste that into the Javascript console and run it, in the server logs you should see the value of "req.body" for the "/hi" endpoint, "{ x: 1 }". Neat, that's what we were looking for. 

If you were wondering why we would ever use GET if POST has the same features but infinite length parameters, it's because GET inherently runs faster than POST. That means that if we know that our parameters will be fixed length, we should aim towards using GET requests instead of POST requests.

That's basically the core of Express.js, there's obviously a lot more to it, but most of that comes from experience and Google searches. Now before we start developing, let's look at tests.

## Tests

How do you know your server code is working? Unlike in the frontend, if your backend code doesn't work, you won't be able to see it immediately. This is why tests are super important for the backend. Inside of the server/tests folder, there's a test file called user.test.js. It should look like this:

In [ ]:
const app = require("../server");
const supertest = require("supertest");

// test our dummy endpoint
test("GET /api/user/sup", async () => {
  await supertest(app)
    .get("/api/user/sup")
    .expect(200)
    .then((response) => {
      // Check type and length
      expect(typeof response.body == "object").toBeTruthy();
      expect(Object.keys(response.body).length).toEqual(1);

      // Check data
      expect(response.body.foo).toBe("bar");
    });
});


We're importing the server app, as well as a npm package called "supertest". You can take a guess at what it's for.

Let's look at the test function, it takes 2 arguments, the first is the name of the test, and the second is a function that defines the test itself. Don't worry about the async/await stuff, just copy and paste that every time you create a new endpoint test. We're calling "supertest(app)", and then using a bunch of dot functions to define how the test should look/behave. 

The first is ".get("/api/user/sup")", which says that the test is making a GET request to the path "/api/user/sup". ".expects(200)" has to do with HTTP response codes, which you can read about here: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status, but all you have to know is that 200 means that the request was successful. 

Now we get to the meat of the test:

In [ ]:
then((response) => {
    // Check type and length
    expect(typeof response.body == "object").toBeTruthy();
    expect(Object.keys(response.body).length).toEqual(1);

    // Check data
    expect(response.body.foo).toBe("bar");
  });

Don't worry too much about the details of this test, just know that response.body stores the JSON response of the endpoint, and "expect(variable).toEqual(value)" is how we will test things. You could just log something to the console to. How do we run these tests?

Open up a new terminal and run "npm run test" in the root folder. If you look in the package.json file, you'll see that this runs "jest server". Jest is a test runner that will run all tests in the folder given (in this case "/server"). A test file is a file that ends in .test.js, like user.test.js, so make sure you name your test files correctly. Jest will give you nicely formatted outputs on which tests passed/failed.

That's how you test GET requests, let's do a test for POST requests now:

In [ ]:
const app = require("../server");
const supertest = require("supertest");

// test our dummy endpoint
test("GET /api/user/sup", async () => {
  await supertest(app)
    .get("/api/user/sup")
    .expect(200)
    .then((response) => {
      // Check type and length
      expect(typeof response.body == "object").toBeTruthy();
      expect(Object.keys(response.body).length).toEqual(1);

      // Check data
      expect(response.body.foo).toBe("bar");
    });
});

// test our dummy endpoint
test("POST /api/user/hi", async () => {
  await supertest(app)
    .post("/api/user/hi")
    .send({x: 1})
    .expect(200)
    .then((response) => {
      console.log(response.body);
    });
});


I added a test for the POST request endpoint we defined earlier, and the test looks similar to the GET request test except we use ".send" to pass in a body. If you run the tests now, you should see how this test ran and all of the logs to the console that were related to each test.

Now that we've gotten a good idea of how our project works/looks, lets add a new endpoint.

## Development

Let's say we wanted to create a server endpoint that takes in a POST request to search for courses. We want endpoints related to courses to be separate from endpoints related to users, so let's create a new file course.js inside of the routes folder where index.js and user.js live. If we copy over some boilerplate code (code that's very simple and is mostly for foundation), we end up with this:

In server/routes/course.js:

In [ ]:
const express = require("express");
const router = express.Router();

router.post("/search", (req, res) => {
    
});


module.exports = router;

Now before we write the code for the endpoint, let's hook it up to our api router in server/routes/index.js:

In [ ]:
const express = require("express");
const userRoute = require("./user");
const courseRoute = require("./course"); // IMPORT IT HERE

// the base router for all api endpoints
const router = express.Router();

// install the user route
router.use("/user", userRoute);
router.use("/course", courseRoute); // INSTALL IT HERE

module.exports = router;

Now how exactly do we know what to do with our endpoint? This is where Technical Design Documents come in handy. They define what our endpoint should look like. This is the specification for the search endpoint (which is highly subject to change):

In [ ]:
// POST /api/course/search
// Body:
// {
// 	“search_query”: str, // the text in the search bar
// 	“course_department”: str,
// 	“course_number”: str, // might be a range, e.g. 100-180B
// 	“course_ge_list”: list of GE strings,
// 	“course_units”: num,
// }

// Response:
// {
//  success: bool,
// 	courseList: list of Course objects
// }

What this specification tells us is that this endpoint is for POST requests, should be on the path /api/course/search, will take in a request body as specified, and will return a response body as specified. Notice that this specification doesn't tell us how exactly the endpoint will do so, it just tells us what it will look like, so let's implement that.

Endpoints are composed of 3 steps:
- Validate arguments
- Core Logic
- Response

The first is to validate arguments. Our server shouldn't assume that the data it's receiving in a request is valid. For now, checking to see if we have the right parameters and that they are the correct types is sufficient.

In [ ]:
const express = require("express");
const router = express.Router();

router.post("/search", (req, res) => {
  if (
    !("search_query" in req.body) ||
    !("department" in req.body) ||
    !("number" in req.body) ||
    !("ge_list" in req.body) ||
    !("units" in req.body)
  ) {
    res.status(400).send({ success: false, message: "Missing arguments" });
    return;
  }
  if (
    !(typeof req.body.search_query === "string") ||
    !(typeof req.body.department === "string") ||
    !(typeof req.body.number === "string") ||
    !Array.isArray(req.body.ge_list) ||
    !(typeof req.body.units === "number")
  ) {
    res.status(400).send({ success: false, message: "Bad argument types" });
  }
});

module.exports = router;


This seems rather tedious, but it will likely save a lot of headaches later on. Notice that I'm using "res.status", which lets us change the status code from the default 200 to whatever we want. 400 means that the request was bad.

Now since we'll be developing the core logic for this endpoint later, we just need to send back some dummy data for now. Look back at the specification to see what format the response should be, but in this case, we have to send back course objects. Course objects are defined at the bottom of the Technical Design Document (and are also subject to change).

In [ ]:
// Course
// {
// 	id: str // in the format : “I&CSCI53”
//  name: str,
// 	department: str,
// 	prerequisites: str,
// 	corequisite: str,
// 	same_as: str,
// 	overlaps: str,
// 	recommended: str,
// 	concurrent: str,
// 	repeatability: str,
// 	grade_option: str,
// 	quarter_offering: str,
// 	restriction: str,
// 	units_min: num,
// 	units_max: num,
// }

Let's make sure we return something like that.

In [ ]:
const express = require("express");
const router = express.Router();

router.post("/search", (req, res) => {
  if (
    !("search_query" in req.body) ||
    !("department" in req.body) ||
    !("number" in req.body) ||
    !("ge_list" in req.body) ||
    !("units" in req.body)
  ) {
    res.status(400).send({ success: false, message: "Missing arguments" });
    return;
  }
  if (
    !(typeof req.body.search_query === "string") ||
    !(typeof req.body.department === "string") ||
    !(typeof req.body.number === "string") ||
    !Array.isArray(req.body.ge_list) ||
    !(typeof req.body.units === "number")
  ) {
    res.status(400).send({ success: false, message: "Bad argument types" });
  }
  res.send({
    success: true,
    courseList: [
      {
        id: "I&CSCI53", // in the format : “I&CSCI53”
        name: "System Design",
        department: "ICS",
        prerequisites: "ICS 51",
        corequisite: "",
        same_as: "",
        overlaps: "",
        recommended: "",
        concurrent: "",
        repeatability: "",
        grade_option: "",
        quarter_offering: "",
        restriction: "",
        units_min: 4,
        units_max: 4,
      },
    ],
  });
});

module.exports = router;


Now how should we check to see if our endpoint works. Tests! Let's create a new test file course.test.js and put this in it:

In [ ]:
const app = require("../server");
const supertest = require("supertest");

test("POST /api/course/search", async () => {
  await supertest(app)
    .post("/api/course/search")
    .send({
      search_query: "ics 51",
      department: "",
      number: "",
      ge_list: ["I", "II"],
      units: 5,
    })
    .expect(200)
    .then((response) => {
      expect(response.body.success).toEqual(true);
      expect(Array.isArray(response.body.courseList)).toEqual(true);
      expect(response.body.courseList.length > 0).toEqual(true);
    });
});


Now notice that I didn't really check the actual values inside of the response, just some generic features about it. That's what our tests should aim for, to be as generic as possible to not have to be updated constantly, but also not so generic that they don't testing anything meaningful. Run the tests and make sure that everything checks out!

That's basically it for backend development, it's a cycle of:
- Looking at the technical design document specifications to see what endpoints should look like
- Implement the endpoint with argument validation, core logic, and response, using dummy data as necessary
- Create tests to check that the endpoint works

And if you couldn't tell from the length of this tutorial, there's a lot to web development, just in the back end alone (the front end is its own beast), so ask questions! Spend some time on it. It'll take a couple of hours just to get one thing in at the beginning, but you'll get better slowly through experience. If you ever want to get on a call and have me walk you through stuff, I'd be happy to. Good luck!